In [1]:
import traceback
import datetime
import yaml

import pytz
from Gradio_UI import GradioUI
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, load_tool, tool

from ollama.smol_compatiable_model import OllamaQwen7BClientModel
from tools.final_answer import FinalAnswerTool

/Users/maksim.rostov/pdev/freestyling/agents/hf-course/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@tool
def search_online(search_query:str, max_results:int) -> str: 
    """A tool that gets info from the internet (via DuckDuckGo) and prints it out in a single string.
    Args:
        search_query: the first argument
        max_results: the second argument
    """
    try:
        resp = DuckDuckGoSearchTool(max_results=max_results).forward(search_query)
    except Exception as e:
        resp = "Could not use the websearch tool, something went wrong: " + traceback.format_exc() # or e.__repr__
    return resp

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"
    
# Tool that is needed to finish agent's work, that's how they understand that they are done 
final_answer = FinalAnswerTool()

In [3]:
# model = InferenceClientModel(
#     max_tokens=2096,
#     temperature=0.5,
#     model_id='Qwen/Qwen2.5-Coder-32B-Instruct',
#     custom_role_conversions=None,
# )
model = OllamaQwen7BClientModel()

# Import tool from Hugging face Hub
image_generation_tool = load_tool("agents-course/text-to-image", trust_remote_code=True)

In [4]:
with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)    
prompt_templates["final_answer"] =  {"{final_answer}"}

print(prompt_templates)

{'system_prompt': 'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using the `final_ans

In [5]:
prompt_templates = {
    "system": {
        "prefix": "You are a coding assistant. Follow rules:\n- Use tools\n- Explain steps.",
        "suffix": ""
    },
    "user": {
        "prefix": "Task: {input}",
        "suffix": ""
    },
    "tool": {
        "prefix": "Action: Run {tool_name} with input: {tool_input}",
        "suffix": ""
    },
    "tool_result": {
        "prefix": "Result: {tool_output}",
        "suffix": ""
    },
    "final_answer": {
        "prefix": "Final Answer: {final_answer}",
        "suffix": ""
    }
    
}

In [6]:
agent = CodeAgent(
    model=model,
    tools=[final_answer, get_current_time_in_timezone, search_online], 
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    # prompt_templates=prompt_templates # Pass system prompt to CodeAgent
)


In [7]:
agent.run("Just get some info about Prussia")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Just get some info about Prussia                                                                                │
│                                                                                                                 │
╰─ OllamaQwen7BClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ─────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

| smolagents/agents.py | func: _step_stream | input_messages =  [{'role': <MessageRole.SYSTEM: 'system'>, 'content': [{'type': 'text', 'text': 'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' 

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  info_on_prussia = search_online(search_query='info on prussia', max_results=5)                                   
  print(info_on_prussia)                                                                                           
                                                                                                                   
  from bs4 import BeautifulSoup                                                                                    
                                                                                                                   
  # Assuming info_on_prussia is an actual dictionary format with HTML content as a string                          
  html_content = info_on_prussia.get('response', '')                                                               
  soup = BeautifulSoup(html_content, 'html.parser')                                                                
                                                                                                                   
  # Remove any HTML tags to get plain text                                                                         
  cleaned_text = soup.get_text()                                                                                   
                                                                                                                   
  # Break down the information into key points                                                                     
  key_points = cleaned_text.split('\n')                                                                            
                                                                                                                   
  summary_of_info_on_prussia = f"The historical region of Prussia had its peak during the late 18th and early      
  19th centuries when it became a state by joining the German Confederation. It consisted predominantly of         
  territories in north-eastern Germany and northern Poland, which were ceded to it by the partitions of Poland at  
  the end of the 18th century."                                                                                    
                                                                                                                   
  final_answer(summary_of_info_on_prussia)                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Prussia | History, Maps, Flag, & Definition | Britannica](https://www.britannica.com/place/Prussia)
The Teutonic Order's last grand master in Prussia, Albert of Hohenzollern, became a Lutheran and, in 1525, 
secularized his fief, which he transformed into a duchy for himself. Thereafter until 1701 this territory (i.e., 
East Prussia) was known as Ducal Prussia.When Albert's son and successor, Albert Frederick, died sonless in 1618, 
the duchy passed to his eldest daughter's husband, the ...

[Prussia - Wikipedia](https://en.wikipedia.org/wiki/Prussia)
Prussia (/ ˈ p r ʌ ʃ ə /; German: Preußen [ˈpʁɔʏsn̩] ⓘ; Old Prussian: Prūsija) was a German state centred on the 
North European Plain that originated from the 1525 secularization of the Prussian part of the State of the Teutonic
Order.The Knights had to relocate their headquarters to Mergentheim, but managed to keep land in Livonia until 
1561.. Prussia formed the German Empire ...

[Kingdom of Prussia - Wikipedia](https://en.wikipedia.org/wiki/Kingdom_of_Prussia)
The Kingdom of Prussia [a] (German: Königreich Preußen, pronounced [ˈkøːnɪkʁaɪç ˈpʁɔʏsn̩] ⓘ) was a German state that
existed from 1701 to 1918. [5] It played a significant role in the unification of Germany in 1871 and was a major 
constituent of the German Empire until its dissolution in 1918. [5] Although it took its name from the region 
called Prussia, it was based in the ...

[No Seriously, What was Prussia? - Circa 
Project](https://www.circaproject.com/historytodayblog/no-seriously-what-was-prussia)
Prussia was a kingdom in central Europe, founded in 1525. Prussia was located where today we would find Germany and
Poland. In the map above, which is from 1806, you can see Prussia in blue, located between Denmark and Austria. 
Prussia was founded at a time when the map of Europe was populated by far more states than it is today.

[Prussia (region) - Wikipedia](https://en.wikipedia.org/wiki/Prussia_(region))
Prussia (Prussian: Prūsa; Polish: Prusy ⓘ; Lithuanian: Prūsija; Russian: Пруссия [ˈprusʲ(ː)ɪjə] ⓘ; German: Preußen 
[ˈpʁɔʏsn̩] ⓘ; Latin: Pruthenia/ Prussia / Borussia) is a historical region in Central Europe on the south-eastern 
coast of the Baltic Sea, that ranges from the Vistula delta in the west to the end of the Curonian Spit in the east
and extends inland as far ...

...

| smolagents/agents.py | func: _step_stream | input_messages =  [{'role': <MessageRole.SYSTEM: 'system'>, 'content': [{'type': 'text', 'text': 'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' 

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  info_on_prussia = """                                                                                            
  Prussia was an 18th-century state founded by King John VIII Palatine, known formally as the Kingdom of Prussia.  
  Its territory covered predominantly north-eastern Germany and northern Poland. The region became strategically   
  important after gaining territories from partitions of Poland at the end of the Partitions of Poland in the      
  late 18th century.                                                                                               
                                                                                                                   
  King Frederick William I formally created the state through the Saxon Convention, following his defeat of        
  Sweden in Battle of Parnawa in 1716.                                                                             
  """                                                                                                              
                                                                                                                   
  summary = info_on_prussia.split('\n')[3] # Getting information about strategy's importance after territorial     
  gain from Poland partitions                                                                                      
  created_info = info_on_prussia.split("\n")[4] # Extracting details regarding creation of the state               
                                                                                                                   
  final_answer({                                                                                                   
      'strategy': summary,                                                                                         
      'creation_details': created_info                                                                             
  })                                                                                                               
                                                                                                                   
  final_answer({                                                                                                   
      'Strategy_Importance': 'The region became strategically important after gaining territories from partitions  
  of Poland at the end of the Partitions of Poland in the late 18th century.',                                     
      'Creation_Details': "King Frederick William I formally created the state through the Saxon Convention,       
  following his defeat of Sweden in Battle of Parnawa in 1716."                                                    
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'strategy': 'King Frederick William I formally created the state through the Saxon Convention,
following his defeat of Sweden in Battle of Parnawa in 1716.', 'creation_details': ''}

[Step 3: Duration 17.31 seconds]

{'strategy': 'King Frederick William I formally created the state through the Saxon Convention, following his defeat of Sweden in Battle of Parnawa in 1716.',
 'creation_details': ''}